<a href="https://colab.research.google.com/github/annabellesauve/Empathy-Detection/blob/main/CISC867_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install tensorflow_addons

     |████████████████████████████████| 2.9 MB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 41.3 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 7.5 MB/s 


In [4]:
import tensorflow as tf
import tensorflow_addons as tfa
from transformers import RobertaTokenizer, TFRobertaModel, TFRobertaForMaskedLM
import pandas as pd
import numpy as np
import math

Roberta documentation: https://huggingface.co/transformers/model_doc/roberta.html?highlight=roberta#tfrobertamodel

# Encoders

In [5]:
# robertabase model initialization, from huggingface
robertabase_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
robertabase_model = TFRobertaModel.from_pretrained('roberta-base')
robertabase_maskedLM_model = TFRobertaForMaskedLM.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at roberta-b

**S Encoder**

In [6]:
# def S_Encoder(seeker_post, id_SP, attention_SP):
class S_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  # def _init_weights(self, module):
  #   # don't know what weights to use to initialize.. not much info 
  #   module.bias.data.zero_()
  #   module.weight.data.fill_(1.0)
  
  def encode(self, seeker_post):
    encoded_input = robertabase_tokenizer(seeker_post, return_tensors='tf')
    return robertabase_model(encoded_input)
    # input_ids = np.array(robertabase_tokenizer.encode(seeker_post, add_special_tokens=True))
    # return robertabase_maskedLM_model(input_ids, labels=input_ids)
    # inputs = robertabase_tokenizer(seeker_post_mask, return_tensors="tf")
    # inputs["labels"] = robertabase_tokenizer(seeker_post, return_tensors="tf")["input_ids"]
    # return robertabase_maskedLM_model(inputs)

**R Encoder**

In [7]:
class R_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  # def _init_weights(self, module):
  #   # don't know what weights to use to initialize.. not much info 
  #   module.bias.data.zero_()
  #   module.weight.data.fill_(1.0)

  def encode(self, response_post):
    encoded_input = robertabase_tokenizer(response_post, return_tensors='tf')
    return robertabase_model(encoded_input)

  # def encode_mask(response_post, response_post_mask):

  #   inputs = robertabase_tokenizer(response_post_mask, return_tensors="tf")["input_ids"]
  #   inputs["labels"] = robertabase_tokenizer(response_post, return_tensors="tf")["input_ids"]
  #   return robertabase_maskedLM_model(inputs)
    

# Attention Layer

In [8]:
class AttentionLayer(tf.keras.Model):
  
  def __init__(self):
    super(AttentionLayer, self).__init__()

  def call(self, e_R, e_S, d):
    a = tf.math.multiply(tf.nn.softmax(tf.math.divide(tf.math.multiply(e_R, e_S), math.sqrt(d))), e_R)
    return a



## dont know where this is used   
def residual(e_R, e_S, d):
  h = e_R + attention(e_R, e_S, d)
  return h

# Models

In [37]:
# BiEncoderAttentionWithRationaleClassification() in models.py
class Model(tf.keras.Model):

  def __init__(self, hidden_dropout_prob=0.2, rationale_num_labels=2, empathy_num_labels= 3, hidden_size=768):
    super(Model, self).__init__()

    self.hidden_dropout_prob = hidden_dropout_prob
    self.rationale_classifier = tf.keras.layers.Dense(rationale_num_labels, activation=None) # linear layer
    self.attention = AttentionLayer()
    self.rationale_num_labels = rationale_num_labels
    self.empathy_num_labels = empathy_num_labels
    self.empathy_classifier = EmpathyClassifier() # linear layer
    self.r_encoder = R_Encoder()
    self.s_encoder = S_Encoder()
    self.hidden_size = hidden_size

  # def _init_weights(self, module):
  #   # don't know what weights to use to initialize.. not much info 
  #   module.bias.data.zero_()
  #   module.weight.data.fill_(1.0)
  
  def call(self,
		input_ids_SP=None,
		input_ids_RP=None,
		attention_mask_SP=None,
		attention_mask_RP=None,
		token_type_ids_SP=None,
		token_type_ids_RP=None,
		position_ids_SP=None,
		position_ids_RP=None,
		head_mask_SP=None,
		head_mask_RP=None,
		inputs_embeds_SP=None,
		inputs_embeds_RP=None,
		empathy_labels=None,
		rationale_labels=None,
		lambda_EI=1,
		lambda_RE=0.1,
    SP=None,
    RP=None
    ):

    output_SP = []
    output_RP = []

    for post in SP:
      output_SP.append(self.s_encoder.encode(post))
    
    for post in RP:
      output_RP.append(self.r_encoder.encode(post))

    sequence_output_SP = output_SP[0].pooler_output
    sequence_output_RP = output_RP[0].pooler_output

    attn = self.attention(sequence_output_RP, sequence_output_SP, self.hidden_size)
    sequence_output_RP = sequence_output_RP + tf.nn.dropout(attn, self.hidden_dropout_prob)

    logits_empathy = self.empathy_classifier(sequence_output_RP)

    sequence_output = tf.nn.dropout(sequence_output_RP, self.hidden_dropout_prob)
    logits_rationales = self.rationale_classifier(sequence_output)

    labels = tf.reshape(empathy_labels, -1)
    logits = tf.reshape(logits_empathy, (-1, self.empathy_num_labels))
    loss_empathy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits).numpy()

    print("loss empathy:", loss_empathy)

    active_loss = tf.reshape(attention_mask_RP, -1)
    active_logits = tf.reshape(logits_rationales, (-1, self.rationale_num_labels))

    if 1 in active_loss: # not sure about this condition, they use == 1
      # active_labels = tf.reshape(rationale_labels, -1)
      active_labels = tf.reshape([1], -1)
    else:
      # active_labels = tf.tensor(-100, shape=-1, dtype=tf.dtype(rationale_labels))
      active_labels = tf.reshape([0], -1)

    loss_rationales = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=active_labels, logits=active_logits).numpy()

    print("loss rationale:", loss_rationales)

    overall_loss = lambda_EI * loss_empathy + lambda_RE + loss_rationales

    return (overall_loss, loss_empathy, loss_rationales, logits_empathy, logits_rationales)

# Empathy Identification

In [10]:
# RobertaClassificationHead in model.py
class EmpathyClassifier(tf.keras.Model):

  def __init__(self, hidden_dropout_prob=0.1, hidden_size=768, empathy_num_labels=3):
    super(EmpathyClassifier, self).__init__()

    self.dense = tf.keras.layers.Dense(hidden_size, activation=None)
    self.hidden_dropout_prob = hidden_dropout_prob
    self.out_proj = tf.keras.layers.Dense(empathy_num_labels, activation=None)
    self.relu = tf.keras.layers.ReLU()

  def call(self, features):
    x = features[:, :]
    x = tf.nn.dropout(x, self.hidden_dropout_prob)
    x = self.dense(x)
    x = self.relu(x)
    x = tf.nn.dropout(x, self.hidden_dropout_prob)
    x = self.out_proj(x)
    return x

# Initializing model + training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# load dataset
  # assuming for now seeker posts is "seeker_posts" and reponse posts is "response_posts"
df_ER = pd.read_csv('/content/drive/My Drive/emotionalreactions.csv')
df_ER = df_ER.head(50) # temporary

df_ER['response_post_masked'] = df_ER['response_post_masked'].replace(np.nan, 0)

In [ ]:
# roberta domain adaptive pre training 3 epoch 8 batch

# can't get this to work. the tfrobertaformaskedlm example doesn't even work.

# optimizer_roberta = tfa.optimizers.AdamW(
#     weight_decay=1e-2, # not given but default in py torch so probably used
#     learning_rate=2e-5, # given
#     epsilon=1e-8 # given
# )

# sp = df_ER.seeker_post 
# rp = df_ER.response_post
# rp_masked = df_ER.response_post_masked

# encoder_train_data = [sp, rp, rp_masked]

# S_encoder_model = S_Encoder()
# R_encoder_model = R_Encoder()

# total_encoder_train_loss = 0
# total_encoder_sp_loss = 0
# total_encoder_rp_loss = 0

# for epoch in range(0, 3):
#   print("Training epoch #", epoch, "...")

#   for i in range(len(encoder_train_data[0])):
#       e_sp = encoder_train_data[0][i] # seeker post 
#       e_rp = encoder_train_data[1][i] # response post 
#       e_rp_masked = encoder_train_data[2][i] # masked response post

#       # sp_output = S_Encoder.encode(e_sp)
#       if not isinstance(e_rp_masked, str):
#         continue 
#       else:
#         rp_output = R_Encoder.encode_mask(e_rp, e_rp_masked)

#         print(rp_output)
#         total_encoder_train_loss += rp_output.loss # what is total loss? no info
#         # total_encoder_sp_loss += sp_output.loss
#         total_encoder_rp_loss += rp_output.loss

#         optimizer_roberta.minimize(rp_output.loss) # update optimizer

#         print(rp_output.loss)
    
#   print("Epoch #", epoch)
#   print("Total loss:", total_train_loss)
#   print("Total empathy loss:", total_train_empathy_loss)
#   print("Total rationale loss:", total_train_rationale_loss)

In [38]:
# tokenize input 
tokenizer_RP = robertabase_tokenizer.batch_encode_plus(df_ER.response_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
input_ids_RP = tokenizer_RP['input_ids']
attention_masks_RP = tokenizer_RP['attention_mask']

tokenizer_SP = robertabase_tokenizer.batch_encode_plus(df_ER.seeker_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
input_ids_SP = tokenizer_SP['input_ids']
attention_masks_SP = tokenizer_RP['attention_mask']

labels = np.array(df_ER.level.values.astype(int))
rationales = df_ER.rationale_labels.values

ER_model = Model()
# IN_model = Model()
# EX_model = Model()

optimizer_model = tfa.optimizers.AdamW(
    weight_decay=1e-2, # not given but default in py torch so probably used
    learning_rate=2e-5, # given
    epsilon=1e-8 # given
)

# ER_model.compile(optimizer=optimizer_model)

In [23]:
# train_dataset = tf.data.Dataset(input_ids_SP, attention_masks_SP, input_ids_RP, attention_masks_RP, labels, rationales)
train_dataset = [input_ids_SP, attention_masks_SP, input_ids_RP, attention_masks_RP, labels, rationales]

# print(labels)
train_size = int(len(train_dataset))

total_steps = train_size * 4 # 4 is epoch
# num_batch = int(train_size / 32) # 32 is batch size

tf.random.set_seed(12) # given seed

In [48]:
for epoch in range(0, 4):
  print("Training epoch #", epoch, "...")
  total_train_loss = 0
  total_train_empathy_loss = 0
  total_train_rationale_loss = 0

  # ER_model.train()

  for i in range(len(train_dataset[0])):
  # for element in train_dataset:
      e_id_SP = train_dataset[0][i] # get seeker id 
      e_attention_SP = train_dataset[1][i] # get seeker attention 
      e_id_RP = train_dataset[2][i] # get response id 
      e_attention_RP = train_dataset[3][i] # get response attention
      e_labels = train_dataset[4][i] # get labels
      # e_rationales = train_dataset[5][i]
      e_rationales = list(map(int, train_dataset[5][i].split(','))) # get rationales, transform to array of ints

      loss, loss_empathy, loss_rationale, logits_empathy, logits_rationale = ER_model(
          input_ids_SP=e_id_SP,
          input_ids_RP=e_id_RP,
          attention_mask_SP=e_attention_SP,
          attention_mask_RP=e_attention_RP,
          empathy_labels=e_labels,
          rationale_labels=e_rationales,
          SP=df_ER.seeker_post,
          RP=df_ER.response_post
          )
      
      total_train_loss += loss
      total_train_empathy_loss += loss_empathy
      total_train_rationale_loss += loss_rationale

      # print(ER_model.trainable_weights)

      optimizer_model.minimize(loss, var_list=ER_model.trainable_weights, tape=tf.GradientTape(persistent=False)) # update optimizer
      # getting 'numpy.ndarray' object has no attribute '_id' error, has to do with tape.
      # if remove tape then error also

      print("loss:", loss)
    
  print("Epoch #", epoch)
  print("Total loss:", total_train_loss)
  print("Total empathy loss:", total_train_empathy_loss)
  print("Total rationale loss:", total_train_rationale_loss)

Training epoch # 0 ...
loss empathy: [0.8937717]
loss rationale: [0.7969197]


AttributeError: ignored